In [2]:
import json

In [28]:
with open('rooms.json', 'r') as file:
    room_data = json.load(file)

In [103]:
with open('students.json', 'r') as file:
    stud_data = json.load(file)

In [39]:
# connection
import psycopg2

#Establishing the connection
conn = psycopg2.connect(
   database="python_intro", 
   user='postgres', 
   password='1', 
   host='localhost', 
   port= '5432'
)
#Setting auto commit false
conn.autocommit = True

#Creating a cursor object using the cursor() method
cursor = conn.cursor()

In [114]:
# insertion room data
try:
    counter = 0
    for i in room_data:
        cursor.execute(f"INSERT INTO rooms VALUES ({i['id']}, '{i['name']}')")
        counter += 1
    print(f'Sucessfully inserted {counter} rows')
except (Exception, psycopg2.Error) as error:
    print("Failed to insert record into mobile table", error)

Sucessfully inserted 1000 rows


In [116]:
# insertion student data in students table and student_room table
try:
    counter = 0
    for i in stud_data:
        cursor.execute(f"INSERT INTO students VALUES ({i['id']}, '{i['name']}', '{i['room']}', '{i['sex']}', '{i['birthday']}')")
        cursor.execute(f"INSERT INTO student_room VALUES ({i['id']}, '{i['room']}')")
        counter += 1
    print(f'Sucessfully inserted {counter} rows')
except (Exception, psycopg2.Error) as error:
    print("Failed to insert record into mobile table", error)

Sucessfully inserted 10000 rows


In [123]:
# list of rooms and count of students in that rooms. 1st query
cursor.execute('''
                SELECT 
                    r.room_id,
                    r.room_name,
                    COUNT(s.student_id) AS count_of_students
                FROM rooms r 
                    LEFT JOIN students s USING(room_id)
                GROUP BY 1 
                ORDER BY 1
                ''')

rooms_stud_count = cursor.fetchall()

# writing to json
with open("query1.json", "w") as file:
    file.write(json.dumps(rooms_stud_count, indent=4))

In [126]:
# top 5 rooms with the smallest avg age of students in the rooms. 2nd query
cursor.execute('''
                SELECT 
                    *
                FROM (
                    SELECT 
                        *,
                        ROW_NUMBER() OVER(ORDER BY avg_stud_age) AS placement
                    FROM (
                    
                        SELECT 
                            r.room_id,
                            r.room_name,
                            ROUND(AVG(EXTRACT(YEAR FROM s.birthday)), 2) :: FLOAT AS avg_stud_age
                        FROM rooms AS r
                            JOIN students AS s USING(room_id) 
                        GROUP BY 1
                ) AS tmp
                    ) AS tmp2
                WHERE placement < 6
                ''')

top_5_small_age = cursor.fetchall()
# writing to json
with open("query2.json", "w") as file:
    file.write(json.dumps(top_5_small_age, indent=4))

In [127]:
# top 5 rooms with the highest difference in age of students in the rooms. 3rd query
cursor.execute('''
                SELECT 
                    *
                FROM (
                    SELECT 
                        *,
                        ROW_NUMBER() OVER(ORDER BY age_diff DESC) AS placement
                    FROM (
                    
                        SELECT 
                            r.room_id,
                            r.room_name,
                            MAX(EXTRACT(YEAR FROM s.birthday)) - MIN(EXTRACT(YEAR FROM s.birthday)) :: FLOAT AS age_diff
                        FROM rooms AS r
                            JOIN students AS s USING(room_id) 
                        GROUP BY 1
                        ) AS tmp
                    ) AS tmp2
                WHERE placement < 6
                ''')

top_5_big_age_diff = cursor.fetchall()

# writing to json
with open("query3.json", "w") as file:
    file.write(json.dumps(top_5_big_age_diff, indent=4))

In [128]:
# list of rooms with different sex in one room. 4th query
cursor.execute('''
                SELECT room_name 
                FROM (
                        SELECT 
                            r.room_id,
                            r.room_name,
                            COUNT(DISTINCT s.sex) AS gender_variety
                        FROM rooms AS r
                            JOIN students AS s USING(room_id) 
                        GROUP BY 1
                        HAVING COUNT(DISTINCT s.sex) > 1 
                    ) AS tmp 
                ''')

diff_gender_room = cursor.fetchall()

# writing to json
with open("query4.json", "w") as file:
    file.write(json.dumps(diff_gender_room, indent=4))